### Imports

In [209]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz
import difflib 
from fuzzywuzzy import process
import time

### Importing inputs
- 3 Mout. Volumes 
- wcvp 
- adding a column with author lists to each dataframe

##### updating author functions

In [210]:
#TODO: move this to index code
def fix_author(authors_field):
    """this function removes the extra space that currently exists
       in the 'author' filed of the index csv files
       INPUT: any value
       OUTPUT: the function returns non string values as is
               and removes last character for string values 
    """
    if not isinstance(authors_field, str) or authors_field[-1] != ' ':
        return authors_field
    return authors_field[:-1]

In [211]:
#TODO: set the list of connectives as an external thing
def split_author(author_str):
    """function that splits the author names ignoring given connectives used
       INPUT: a string (of authors)
       OUTPUT: a list of containing each author from the input string
    """
    if (not isinstance(author_str, str)) or author_str == "np.NaN":
        return author_str
    paran_list = []
    reg_list = []

    in_pran = ""
    out_pran = ""
    #the paranthesis part (assuming we have one paranthesis only)
    search_full_paran = re.search(r"\(.*\)", author_str)
    search_close_paran = re.search(r".*\)", author_str)
    if search_full_paran:
        in_pran = search_full_paran.group(0)[1:-1] 
    elif search_close_paran:
        in_pran = search_close_paran.group(0)[:-1]
    
    split_paran = re.split(r".*\)", author_str, maxsplit=1)
    out_pran = split_paran[int(len(split_paran) == 2)]
    
    #remove anything after ex. 
    in_pran = re.sub(r"\sex[\s|.].*","",in_pran)
    out_pran = re.sub(r"\sex[\s|.].*","",out_pran)


    #ignoring et, in, non, &, er, nec, with space or . after (TODO: check end/ beggining of string)
    ignore_pattern = r"\set[\s|.]|\sin[\s|.]|\snon[\s|.]|\s&[\s|.]|\ser[\s|.]|\snec[\s|.]|\smult[\s|.]"
    paran_list = re.split(ignore_pattern, in_pran) 
    reg_list = re.split(ignore_pattern, out_pran) 
    
    #TODO: improve this part
    return [aut for aut in paran_list + reg_list if aut != ""] 

In [212]:
# a function for flagging authors with auct. / agg. etc
# so that we can just use the wcvp version later
# note, vol3 only has 2 occurances of auct. 
# the rest don't show up because we are only looking at non-italicized part of index

##### importing dataframes 

In [213]:
#mout. books: 
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#changing name of columns of mout. indecies 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

##### applying the author functions

In [214]:
#fix spacing issue in Mout. books
vol1_df['mouterde_author'] = vol1_df['mouterde_author'].apply(fix_author)
vol2_df['mouterde_author'] = vol2_df['mouterde_author'].apply(fix_author)
vol3_df['mouterde_author'] = vol3_df['mouterde_author'].apply(fix_author)

#getting author lists
wcvp_df['author_list'] = wcvp_df['authors'].apply(split_author)
vol1_df['mouterde_author_list'] = vol1_df['mouterde_author'].apply(split_author)
vol2_df['mouterde_author_list'] = vol2_df['mouterde_author'].apply(split_author)
vol3_df['mouterde_author_list'] = vol3_df['mouterde_author'].apply(split_author)

### Finding closest Match for the genera

In [215]:
#uses default matching criteria of difflib
def difflib_closest_matches(input_str, matching_lst):
    matched_list = difflib.get_close_matches(input_str, matching_lst)
    if len(matched_list) > 0:
        return matched_list[0]
    else:
        print(input_str)
        return np.NaN

In [216]:
wcvp_unique_genera = wcvp_df['genus'].unique()


startTime = time.time()
#prints the ones with no closest match
print("---- vol1 genus match ----")
vol1_df['mout_genus_close_match'] = vol1_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol2 genus match ----")
vol2_df['mout_genus_close_match'] = vol2_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol3 genus match ----")
vol3_df['mout_genus_close_match'] = vol3_df['mouterde_genus'].apply(lambda x: difflib_closest_matches(x, wcvp_unique_genera))
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

---- vol1 genus match ----
j.
j.
j.
j.
j.
j.
j.
j.
Seconds to Run:119.27695798873901
-------- complete --------

---- vol2 genus match ----
Seconds to Run:130.5730311870575
-------- complete --------

---- vol3 genus match ----
Seconds to Run:153.86549305915833
-------- complete --------



In [217]:
print(vol1_df[vol1_df['mout_genus_close_match'] != vol1_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol1_df[vol1_df['mout_genus_close_match'] != vol1_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

25


,mouterde_genus,mout_genus_close_match
31,AgropyrQn,Agropyron
214,Asphodelusi,Asphodelus
282,Bœrhavia,Boerhavia
320,Buffonia,Bufonia
372,Gerastium,Cerastium
383,Geterach,Ceterach
453,Çutandia,Cutandia
496,Danaë,Diana
544,Eremopyron,Eremophyton
618,filicaulis,Silicularia


In [218]:
print(vol2_df[vol2_df['mout_genus_close_match'] != vol2_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol2_df[vol2_df['mout_genus_close_match'] != vol2_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

27


,mouterde_genus,mout_genus_close_match
105,xanthocarpum,Acanthocarpus
118,Ampélopsis,Ampelopsis
143,Anisociadium,Anisosciadium
313,Àubrietia,Aubrieta
357,Buffonia,Bufonia
414,Cardariâ,Cardaria
436,•Ceratophyllum,Ceratophyllum
451,Ghesneya,Chesneya
460,Chrysocamela,Chrysochamela
466,Gircaea,Dircaea


In [219]:
print(vol3_df[vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol3_df[vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

34


,mouterde_genus,mout_genus_close_match
71,Anthémis,Anthemis
77,corymbulosa,Corymbula
78,cotula,Rotula
79,eretica,Herpetica
82,didymaea,Didymaea
83,halophila,Halophila
84,haussknechtii,Haussknechtia
85,hebronica,Veronica
86,hyalina,Thyana
87,leucanthemifolia,Leucanthemella


In [220]:
# only the non-lower case ones for vol3: 
print(vol3_df[(vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']) & ~(vol3_df['mouterde_genus'].str.islower())][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_genus_close_match'] != vol3_df['mouterde_genus']) & ~(vol3_df['mouterde_genus'].str.islower())][['mouterde_genus', 'mout_genus_close_match']].drop_duplicates()

12


,mouterde_genus,mout_genus_close_match
71,Anthémis,Anthemis
157,BallotaL.,Ballota
209,Calhpeltis,Callipeltis
424,CordiaL.,Cordia
438,Crépis,Crepis
511,Flore,Floresia
559,Engeron,Erigeron
701,Hypochoeris,Hypochaeris
882,Moriria,Morierina
1188,Satureia,Saturnia


for volume3, use 'genus_matched' (includes both accepted matches and genera in vol3_accepted_genus_changes) to see the closest match of which genera should be used

In [221]:
vol1_accepted_genus_changes = {}
vol2_accepted_genus_changes = {}

vol3_accepted_genus_changes = {
    "Anthémis" : "Anthemis",
    "BallotaL." : "Ballota",
    "CordiaL." : "Cordia",
    "Crépis" : "Crepis",
    "SolidagoL." : "Solidago"
}

In [222]:
vol1_df['genus_matched'] = (vol1_df['mout_genus_close_match'] == vol1_df['mouterde_genus']) | (vol1_df['mouterde_genus'].isin(vol1_accepted_genus_changes))
vol2_df['genus_matched'] = (vol2_df['mout_genus_close_match'] == vol2_df['mouterde_genus']) | (vol2_df['mouterde_genus'].isin(vol2_accepted_genus_changes))
vol3_df['genus_matched'] = (vol3_df['mout_genus_close_match'] == vol3_df['mouterde_genus']) | (vol3_df['mouterde_genus'].isin(vol3_accepted_genus_changes))

In [330]:
vol1_df

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank,mouterde_author_list,mout_genus_close_match,genus_matched,mout_epithet_close_match,mout_infra_close_match
0,616,Acanthophyllum,NaN,NaN,NaN,genus,NaN,Acanthophyllum,True,NaN,NaN
1,616,Acanthophyllum,kurdicum,NaN,Boiss. et Hausskn.,species,"[Boiss., Hausskn.]",Acanthophyllum,True,kurdicum,NaN
2,616,Achyranthes,NaN,NaN,NaN,genus,NaN,Achyranthes,True,NaN,NaN
3,616,Achyranthes,aspera,NaN,L.,species,[L.],Achyranthes,True,aspera,NaN
4,616,Adiantum,NaN,NaN,NaN,genus,NaN,Adiantum,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1382,638,Vulpia,uniglumis,NaN,(Sol.) Dumort.,species,"[Sol., Dumort.]",Vulpia,True,uniglumis,NaN
1383,638,Zannichellia,NaN,NaN,NaN,genus,NaN,Zannichellia,True,NaN,NaN
1384,638,Zannichellia,palustris,NaN,L.,species,[L.],Zannichellia,True,palustris,NaN
1385,638,Zostera,NaN,NaN,NaN,genus,NaN,Zostera,True,NaN,NaN


### For each species, find the closes match within it's closest match genera

In [223]:
#uses default matching criteria of difflib
def difflib_closest_epithet_match(row):
    epithet = row['mouterde_epithet']
    if row['genus_matched'] and isinstance(epithet, str):
        genus = row['mout_genus_close_match']
        matching_list = wcvp_df[wcvp_df['genus'] == genus]['species'].unique()
        cleanedList = [x for x in matching_list if str(x) != 'nan']
        match_result = difflib.get_close_matches(epithet, cleanedList)  
        
        if match_result:
            return match_result[0]
        else:
            return np.NaN
    else: 
        return np.NaN

In [224]:
startTime = time.time()
#prints the ones with no closest match
print("---- vol1 epithet match ----")
vol1_df['mout_epithet_close_match'] = vol1_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

startTime = time.time()
print("---- vol2 epithet match ----")
vol2_df['mout_epithet_close_match'] = vol2_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

startTime = time.time()
print("---- vol3 epithet match ----")
vol3_df['mout_epithet_close_match'] = vol3_df.apply(difflib_closest_epithet_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("--------- complete ---------\n")

---- vol1 epithet match ----
Seconds to Run:86.29983520507812
--------- complete ---------

---- vol2 epithet match ----
Seconds to Run:101.18236112594604
--------- complete ---------

---- vol3 epithet match ----
Seconds to Run:58.99151086807251
--------- complete ---------



##### Places where non NaN matches of the epithet do not overlap and the number of such occurances

In [225]:
print(vol1_df[(vol1_df['mout_epithet_close_match'] != vol1_df['mouterde_epithet']) & ~(vol1_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol1_df[(vol1_df['mout_epithet_close_match'] != vol1_df['mouterde_epithet']) & ~(vol1_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

91


,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
12,Aegilops,juvenali∫,juvenalis
17,Aegilops,peregrina(Hack.),peregrina
22,Aegilops,umbeUulata,umbellulata
27,Aellenia,glauc,glauca
71,Allium,de∫er˛i,deserti
...,...,...,...
1268,Spergularia,bocconii,bocconei
1279,Spirodela,polyrrhiza,polyrhiza
1328,Trisetaria,kœlerioides,koelerioides
1336,Tulipa,auCheriana,aucheriana


In [226]:
print(vol2_df[(vol2_df['mout_epithet_close_match'] != vol2_df['mouterde_epithet']) & ~(vol2_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol2_df[(vol2_df['mout_epithet_close_match'] != vol2_df['mouterde_epithet']) & ~(vol2_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

107


,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
6,Acer,tauricolum,tauricum
58,Alchemilla,noribunda,floribunda
62,Alhagi,pseudalhâgi,pseudalhagi
74,Althaea,bertrami,bertramii
98,Alyssum,stapffii,stapfii
...,...,...,...
1389,Polypogon,semiyerticillatus,semiverticillatus
1425,Psammogeton,setifolium,setifolius
1432,Pteris,vïttata,vittata
1451,Ranunculus,macrorrhynchup,macrorrhynchus


In [227]:
print(vol3_df[(vol3_df['mout_epithet_close_match'] != vol3_df['mouterde_epithet']) & ~(vol3_df['mout_epithet_close_match'].isna())][['mout_genus_close_match','mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_epithet_close_match'] != vol3_df['mouterde_epithet']) & ~(vol3_df['mout_epithet_close_match'].isna())][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

41


,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
126,Artemisia,verlotorum,verlotiorum
156,Atropa,belladonna,bella-donna
320,Centaurea,ptosimoppapa,ptosimopappa
347,Centranthus,longinorus,longiflorus
462,Crepis,zacyntha,zacintha
464,Cressa,eretica,cretica
582,Fontanesia,phillyreoides,philliraeoides
640,Gnaphalium,luteo-album,luteoalbum
651,Hedypnois,eretica,cretica
683,Hieracium,bauhinii,bauhini


##### Closest match for genus exists
##### but closest match for epithet doesn't

In [228]:
vol1_df[~(vol1_df['mouterde_epithet'].isna()) & (vol1_df['mout_epithet_close_match'].isna()) & (vol1_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
25,Aegilotricum,loreti,NaN
550,Eremurus,hostii,NaN
551,Eremurus,ravennae,NaN
650,Heleocharis,macrantha,NaN
651,Heleocharis,palustris,NaN
698,Hypocylix,kerneri,NaN
1267,Spergularia,et,NaN


In [229]:
vol2_df[~(vol2_df['mouterde_epithet'].isna()) & (vol2_df['mout_epithet_close_match'].isna()) & (vol2_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
393,Callitriche,sp.,NaN
396,Calycotome,villosa,NaN
433,Ceratocephalus,falcafus,NaN
504,Consolida,flâva,NaN
667,Erucaria,boveana,NaN
982,Iberis,sofarana,NaN
1069,Linum,corymbulosum(Planch.)Reichenb.,NaN
1074,Linum,sulphureurn,NaN


In [230]:
vol3_df[~(vol3_df['mouterde_epithet'].isna()) & (vol3_df['mout_epithet_close_match'].isna()) & (vol3_df['genus_matched'] == True)][['mout_genus_close_match', 'mouterde_epithet', 'mout_epithet_close_match']].drop_duplicates()

,mout_genus_close_match,mouterde_epithet,mout_epithet_close_match
631,Garhadiolus,et,NaN
689,Hormuzakia,aggregata,NaN
888,Myopordon,genitz,NaN
930,Odontites,hitea,NaN
1038,Phyllocara,aucheri,NaN
1097,Psychrogeton,son,NaN


### Finding closest infra

In [231]:
#uses default matching criteria of difflib
def difflib_closest_infra_match(row):
    infra = row['mouterde_infra']
    epithet = row['mout_epithet_close_match'] 
    genus = row['mout_genus_close_match']
    if isinstance(infra, str) and row['genus_matched'] and row['mout_epithet_close_match']:
        matching_list = wcvp_df[(wcvp_df['genus'] == genus)  & (wcvp_df['species'] == epithet)]['infraspecies'].unique()
        cleanedList = [x for x in matching_list if str(x) != 'nan']
        match_result = difflib.get_close_matches(infra, cleanedList)  
        
        if match_result:
            return match_result[0]
        else:
            return np.NaN
    else: 
        return np.NaN

In [232]:
startTime = time.time()
#prints the ones with no closest match
print("---- vol1 infra match ----")
vol1_df['mout_infra_close_match'] = vol1_df.apply(difflib_closest_infra_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol2 infra match ----")
vol2_df['mout_infra_close_match'] = vol2_df.apply(difflib_closest_infra_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

startTime = time.time()
print("---- vol3 infra match ----")
vol3_df['mout_infra_close_match'] = vol3_df.apply(difflib_closest_infra_match, axis=1)
print("Seconds to Run:" + str(time.time() - startTime))
print("-------- complete --------\n")

---- vol1 infra match ----
Seconds to Run:0.010306119918823242
-------- complete --------

---- vol2 infra match ----
Seconds to Run:0.24144697189331055
-------- complete --------

---- vol3 infra match ----
Seconds to Run:13.38283395767212
-------- complete --------



In [233]:
print(vol3_df[(vol3_df['mout_infra_close_match'] != vol3_df['mouterde_infra']) & ~(vol3_df['mouterde_infra'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_infra_close_match'] != vol3_df['mouterde_infra']) & ~(vol3_df['mouterde_infra'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates()

89


,mout_genus_close_match,mout_epithet_close_match,mouterde_infra,mout_infra_close_match
61,Anchusa,strigosa,mesopotamica,NaN
104,Antirrhinum,majus,angustifolium,glutinosum
112,Arctium,minus,minus,NaN
135,Asperula,glomerata,capitata,NaN
149,Asyneuma,rigidum,sinaï,sinai
...,...,...,...,...
1414,Teucrium,polium,mollissimum,NaN
1448,Tragopogon,buphthalmoides,buphthalmoides,NaN
1449,Tragopogon,buphthalmoides,latifolius,NaN
1550,Veronica,anagallis-aquatica,michauxii,NaN


In [234]:
print(vol3_df[(vol3_df['mout_infra_close_match'] != vol3_df['mouterde_infra']) & ~(vol3_df['mout_infra_close_match'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_infra_close_match'] != vol3_df['mouterde_infra']) & ~(vol3_df['mout_infra_close_match'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates()

13


,mout_genus_close_match,mout_epithet_close_match,mouterde_infra,mout_infra_close_match
104,Antirrhinum,majus,angustifolium,glutinosum
149,Asyneuma,rigidum,sinaï,sinai
253,Carduus,pycnocephalus,arabicus,albidus
348,Centranthus,longiflorus,latifolius,atlanticus
388,Cirsium,creticum,gaiUardotii,gaillardotii
659,Helichrysum,armenium,armenium,araxinum
1003,Parentucellia,latifolia,flaviflora,albiflora
1107,Ptilostemon,diacanthus,diacantha,diacanthus
1129,Rhododendron,ponticum,brachycarpum,brachystamineum
1308,Solanum,nigrum,ochroleucum,chlorocarpum


In [235]:
print(vol3_df[(vol3_df['mout_infra_close_match'] == vol3_df['mouterde_infra']) & ~(vol3_df['mout_infra_close_match'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates().shape[0])
vol3_df[(vol3_df['mout_infra_close_match'] == vol3_df['mouterde_infra']) & ~(vol3_df['mout_infra_close_match'].isna())][['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mout_infra_close_match']].drop_duplicates()

87


,mout_genus_close_match,mout_epithet_close_match,mouterde_infra,mout_infra_close_match
14,Achillea,aleppica,aleppica,aleppica
40,Alkanna,orientalis,integrifolia,integrifolia
113,Arctium,minus,pubens,pubens
161,Ballota,nigra,uncinata,uncinata
174,Bidens,pilosa,radiata,radiata
...,...,...,...,...
1568,Veronica,macrostachya,macrostachya,macrostachya
1571,Veronica,panormitana,baradostensis,baradostensis
1573,Veronica,panormitana,panormitana,panormitana
1605,Xanthium,strumarium,strumarium,strumarium


### merge by genus, epithet, infra 

for now I'm not going to match based on mout_infra_close_match 
just:

mout_genus_close_match and mout_epithet_close_match

(if genus_matched == True)

In [236]:
vol1_df

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank,mouterde_author_list,mout_genus_close_match,genus_matched,mout_epithet_close_match,mout_infra_close_match
0,616,Acanthophyllum,NaN,NaN,NaN,genus,NaN,Acanthophyllum,True,NaN,NaN
1,616,Acanthophyllum,kurdicum,NaN,Boiss. et Hausskn.,species,"[Boiss., Hausskn.]",Acanthophyllum,True,kurdicum,NaN
2,616,Achyranthes,NaN,NaN,NaN,genus,NaN,Achyranthes,True,NaN,NaN
3,616,Achyranthes,aspera,NaN,L.,species,[L.],Achyranthes,True,aspera,NaN
4,616,Adiantum,NaN,NaN,NaN,genus,NaN,Adiantum,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1382,638,Vulpia,uniglumis,NaN,(Sol.) Dumort.,species,"[Sol., Dumort.]",Vulpia,True,uniglumis,NaN
1383,638,Zannichellia,NaN,NaN,NaN,genus,NaN,Zannichellia,True,NaN,NaN
1384,638,Zannichellia,palustris,NaN,L.,species,[L.],Zannichellia,True,palustris,NaN
1385,638,Zostera,NaN,NaN,NaN,genus,NaN,Zostera,True,NaN,NaN


In [237]:
vol1_df

,page_num,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon rank,mouterde_author_list,mout_genus_close_match,genus_matched,mout_epithet_close_match,mout_infra_close_match
0,616,Acanthophyllum,NaN,NaN,NaN,genus,NaN,Acanthophyllum,True,NaN,NaN
1,616,Acanthophyllum,kurdicum,NaN,Boiss. et Hausskn.,species,"[Boiss., Hausskn.]",Acanthophyllum,True,kurdicum,NaN
2,616,Achyranthes,NaN,NaN,NaN,genus,NaN,Achyranthes,True,NaN,NaN
3,616,Achyranthes,aspera,NaN,L.,species,[L.],Achyranthes,True,aspera,NaN
4,616,Adiantum,NaN,NaN,NaN,genus,NaN,Adiantum,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1382,638,Vulpia,uniglumis,NaN,(Sol.) Dumort.,species,"[Sol., Dumort.]",Vulpia,True,uniglumis,NaN
1383,638,Zannichellia,NaN,NaN,NaN,genus,NaN,Zannichellia,True,NaN,NaN
1384,638,Zannichellia,palustris,NaN,L.,species,[L.],Zannichellia,True,palustris,NaN
1385,638,Zostera,NaN,NaN,NaN,genus,NaN,Zostera,True,NaN,NaN


In [265]:
vol1_genus_matched_df = vol1_df[vol1_df['genus_matched'] == True].copy()
vol2_genus_matched_df = vol2_df[vol2_df['genus_matched'] == True].copy()
vol3_genus_matched_df = vol3_df[vol3_df['genus_matched'] == True].copy()

vol1_genus_matched_df.replace(np.nan, 'np.NaN', inplace=True)
vol2_genus_matched_df.replace(np.nan, 'np.NaN', inplace=True)
vol3_genus_matched_df.replace(np.nan, 'np.NaN', inplace=True)

wcvp_merge_copy = wcvp_df.copy()
wcvp_merge_copy.replace(np.nan, 'np.NaN', inplace=True)

In [286]:
vol1_merge_df = pd.merge(vol1_genus_matched_df, 
                         wcvp_merge_copy, 
                         how='inner', 
                         left_on=['mout_genus_close_match','mout_epithet_close_match', 'mouterde_infra'], 
                         right_on = ['genus','species', 'infraspecies'])

vol1_merge_simplified = vol1_merge_df[['mouterde_genus',
                                       'mouterde_epithet', 
                                       'mouterde_infra', 
                                       'mouterde_author', 
                                       'mouterde_author_list', 
                                       'author_list', 
                                       'authors', 
                                       'genus',
                                       'species'
                                       ]]

In [298]:
vol2_merge_df = pd.merge(vol2_genus_matched_df, 
                         wcvp_merge_copy, 
                         how='inner', 
                         left_on=['mout_genus_close_match','mout_epithet_close_match', 'mouterde_infra'], 
                         right_on = ['genus','species', 'infraspecies'])

vol2_merge_simplified = vol2_merge_df[['mouterde_genus',
                                       'mouterde_epithet', 
                                       'mouterde_infra', 
                                       'mouterde_author', 
                                       'mouterde_author_list', 
                                       'author_list', 
                                       'authors', 
                                       'genus',
                                       'species'
                                       ]]

In [299]:
vol3_merge_df = pd.merge(vol3_genus_matched_df, 
                         wcvp_merge_copy, 
                         how='inner', 
                         left_on=['mout_genus_close_match','mout_epithet_close_match', 'mouterde_infra'], 
                         right_on = ['genus','species', 'infraspecies'])

vol3_merge_simplified = vol3_merge_df[['mouterde_genus',
                                       'mouterde_epithet', 
                                       'mouterde_infra', 
                                       'mouterde_author', 
                                       'mouterde_author_list', 
                                       'author_list', 
                                       'authors', 
                                       'genus',
                                       'species'
                                       ]]

### Matching authors

#### functions for author overlap and types

In [332]:
def get_num_overlap(row):
    if row['mouterde_author_list'] == "np.NaN" or row['author_list'] == "np.NaN":
        return -1

    authors_set_src = row['mouterde_author_list'] #not a set to account for (L.) L. situation
    authors_set_dst = set(row['author_list'])
    num_src = 0
    for author in authors_set_src: 
        _, score = process.extract(author, authors_set_dst)[0]
        if score > 85:
            num_src += 1
    return num_src

In [333]:
def get_match_type(row):
    authors_set_src = row['mouterde_author_list'] #not a set to account for (L.) L. situation
    authors_set_dst = set(row['author_list'])
    num_overlap = row['num_author_overlap']
    
    if row['mouterde_author_list'] == "np.NaN" and row['author_list'] == "np.NaN" :
        return "No author"

    elif row['mouterde_author_list'] == "np.NaN":
        return "No Mout. author"
    
    elif row['author_list'] == "np.NaN":
        return "No WCVP author"
        
    type = "NO TYPE"
    if num_overlap == 0 and len(authors_set_src) > 0 and  len(authors_set_dst) > 0:
        type = "type -1"
    elif num_overlap == len(authors_set_src) and len(authors_set_src) == len(authors_set_dst):
        type = "type 0" #everything the same
    elif num_overlap == len(authors_set_src):
        type = "type 1" #everything in src match but dst set has more 
    elif num_overlap == len(authors_set_dst):
        type = "type 2" #everything in dst match but src has more 

    elif num_overlap < min(len(authors_set_src), len(authors_set_dst)):
        type = "type 3" # a subset of dst or src do not overlap (and one is not a proper subset of another)
    else:
        print(num_overlap, authors_set_src, authors_set_dst)
    return type

In [334]:
vol1_merge_simplified["num_author_overlap"] = vol1_merge_simplified.apply(get_num_overlap, axis=1)
vol1_merge_simplified["author_match_type"] = vol1_merge_simplified.apply(get_match_type, axis=1)

/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_56919/99236302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol1_merge_simplified["num_author_overlap"] = vol1_merge_simplified.apply(get_num_overlap, axis=1)
/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_56919/99236302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol1_merge_simplified["author_match_type"] = vol1_merge_simplified.apply(get_match_type, axis=1)


In [335]:
vol2_merge_simplified["num_author_overlap"] = vol2_merge_simplified.apply(get_num_overlap, axis=1)
vol2_merge_simplified["author_match_type"] = vol2_merge_simplified.apply(get_match_type, axis=1)

/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_56919/1524690916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol2_merge_simplified["num_author_overlap"] = vol2_merge_simplified.apply(get_num_overlap, axis=1)
/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_56919/1524690916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol2_merge_simplified["author_match_type"] = vol2_merge_simplified.apply(get_match_type, axis=1)


In [336]:
vol3_merge_simplified["num_author_overlap"] = vol3_merge_simplified.apply(get_num_overlap, axis=1)
vol3_merge_simplified["author_match_type"] = vol3_merge_simplified.apply(get_match_type, axis=1)

2 ['Sm.', 'Sibth.', 'Sm.'] {'Sm.'}
2 ['Boiss.', 'Reut.', ' Boiss.'] {'Boiss.'}
2 ['Sm.', 'Sibth.', 'Sm.'] {'Sm.'}


/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_56919/1213449415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol3_merge_simplified["num_author_overlap"] = vol3_merge_simplified.apply(get_num_overlap, axis=1)
/var/folders/gg/rd0htpr953n8c6s3hcyh70kw0000gn/T/ipykernel_56919/1213449415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vol3_merge_simplified["author_match_type"] = vol3_merge_simplified.apply(get_match_type, axis=1)


2 {'Reut.', 'Boiss.', ' Boiss.'} {'Boiss.'} -> goes to else because of this ... NO TYPE no longer needed :)

#### group rows with overlapping genus, epithet, infra, author by best author match with wcvp


In [343]:
#vol1_merge_simplified = vol1_merge_simplified.loc[vol1_merge_simplified.groupby(['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]#[['bookid', 'conceptid', 'weight']]
vol1_merge_simplified_grouped = vol1_merge_simplified.loc[vol1_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]
vol2_merge_simplified_grouped = vol2_merge_simplified.loc[vol2_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]
vol3_merge_simplified_grouped = vol3_merge_simplified.loc[vol3_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]

#### Playing around with different author matchings

##### Type -1 (no match but authors available)

In [444]:
vol1_merge_simplified_grouped[vol1_merge_simplified_grouped['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
29,Aegilotricum,loreti,np.NaN,(Richt.) Maire et Weiller,"[Richt., Maire, Weiller]",[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,0,type -1
48,Alisma,gramineum,np.NaN,Gmel.,[Gmel.],[Lej.],Lej.,Alisma,gramineum,0,type -1
49,Alisma,graminifolium,np.NaN,Ehrh.,[Ehrh.],[Glück],Glück,Alisma,graminifolium,0,type -1
50,Alisma,lanceolatum,np.NaN,Whit.,[Whit.],[With.],With.,Alisma,lanceolatum,0,type -1
60,Allium,aschersonianum,np.NaN,W. Barb.,[W. Barb.],[Barbey],Barbey,Allium,aschersonianum,0,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1405,Tulipa,agenensis,np.NaN,D.C.,[D.C.],[Redouté],Redouté,Tulipa,agenensis,0,type -1
1406,Tulipa,aleppensis,np.NaN,Regel,[Regel],[Boiss.],Boiss. ex Regel,Tulipa,aleppensis,0,type -1
1408,Tulipa,biflora,np.NaN,Pali.,[Pali.],[Pall.],Pall.,Tulipa,biflora,0,type -1
1410,Tulipa,montana,np.NaN,Lindley,[Lindley],[Lindl.],Lindl.,Tulipa,montana,0,type -1


In [445]:
vol2_merge_simplified_grouped[vol2_merge_simplified_grouped['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
8,Acer,tauricolum,np.NaN,Boiss. et Bal.,"[Boiss., Bal.]",[Dippel],Dippel,Acer,tauricum,0,type -1
24,Adonis,microcarpa,np.NaN,D.C.,[D.C.],[DC.],DC.,Adonis,microcarpa,0,type -1
34,Aethionema,coridifolium,np.NaN,D.C.,[D.C.],[DC.],DC.,Aethionema,coridifolium,0,type -1
37,Aethionema,oppositifolium,np.NaN,(Lab.) Boiss.,"[Lab., Boiss.]","[Pers., Hedge]",(Pers.) Hedge,Aethionema,oppositifolium,0,type -1
67,Alhagi,maurorum,np.NaN,Willd.,[Willd.],[Medik.],Medik.,Alhagi,maurorum,0,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1557,Ranunculus,cornutus,np.NaN,D.C.,[D.C.],[Boiss.],Boiss.,Ranunculus,cornutus,0,type -1
1563,Ranunculus,demissus,np.NaN,D.C.,[D.C.],[DC.],DC.,Ranunculus,demissus,0,type -1
1565,Ranunculus,lateriflorus,np.NaN,D.C.,[D.C.],[DC.],DC.,Ranunculus,lateriflorus,0,type -1
1575,Ranunculus,orbiculatus,np.NaN,Boiss. et Bo.,"[Boiss., Bo.]",[Blanche],Blanche,Ranunculus,orbiculatus,0,type -1


In [446]:
vol3_merge_simplified_grouped[vol3_merge_simplified_grouped['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
23,Achillea,wilhelmsii,np.NaN,C. Koch,[C. Koch],[K.Koch],K.Koch,Achillea,wilhelmsii,0,type -1
64,Anchusa,milleri,np.NaN,Willd.,[Willd.],[Lam.],Lam. ex Spreng.,Anchusa,milleri,0,type -1
65,Anchusa,strigosa,np.NaN,Labili.,[Labili.],[Labill.],Labill.,Anchusa,strigosa,0,type -1
86,Anthémis,chia,np.NaN,L. cornucopiae Boiss.,[L. cornucopiae Boiss.],[L.],L.,Anthemis,chia,0,type -1
126,Asteriscus,np.NaN,np.NaN,Miller,[Miller],[Moench],Moench,Asteriscus,np.NaN,0,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1599,Verbascum,oreophilum,np.NaN,C. Koch,[C. Koch],[K.Koch],K.Koch,Verbascum,oreophilum,0,type -1
1606,Verbascum,ported,np.NaN,Post,[Post],[Willk.],Willk.,Verbascum,portae,0,type -1
1611,Verbascum,sinuatum,np.NaN,L. tripolitanum Boiss.,[L. tripolitanum Boiss.],[Hablitz],Hablitz,Verbascum,sinuatum,0,type -1
1638,Veronica,non,np.NaN,Vahl,[Vahl],[Boiss.],Boiss.,Veronica,noeana,0,type -1


In [449]:
print(vol1_merge_simplified[vol1_merge_simplified['author_match_type'] == 'type -1'].shape[0],
      vol2_merge_simplified[vol2_merge_simplified['author_match_type'] == 'type -1'].shape[0],
      vol3_merge_simplified[vol3_merge_simplified['author_match_type'] == 'type -1'].shape[0])

252 264 274


In [451]:
genus = "Anchusa"#"Achillea"
epithet = "milleri"#"aleppica"
infra = "np.NaN"#"aleppica"
vol3_merge_simplified[(vol3_merge_simplified['mouterde_genus'] == genus) & (vol3_merge_simplified['mouterde_epithet'] == epithet) & (vol3_merge_simplified['mouterde_infra'] == infra)]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
64,Anchusa,milleri,np.NaN,Willd.,[Willd.],[Lam.],Lam. ex Spreng.,Anchusa,milleri,0,type -1


In [453]:
genus = "Anchusa"#"Achillea"
epithet = "milleri"#"aleppica"
infra = "np.NaN"#"aleppica"
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'].isna())]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
288711,927538-1,Boraginaceae,Anchusa,milleri,NaN,Anchusa milleri,Lam. ex Spreng.,SPECIES,Accepted,NaN,NaN,NaN,4451-1,Anchusa,L.,In review,"Bot. Gart. Halle, Nachtr. 1: 11 (1801)",NaN,[Lam.]


##### untitled

In [344]:
vol3_merge_simplified[(vol3_merge_simplified["author_match_type"] == "No author") |
                      (vol3_merge_simplified["author_match_type"] == "No Mout. author") |
                      (vol3_merge_simplified["author_match_type"] == 'No WCVP author') 
                     ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
141,BallotaL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Ballota,np.NaN,-1,No Mout. author
268,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
278,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
299,Centaurea,bruguieriana,bruguieriana,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,bruguieriana,-1,No author
300,Centaurea,calcitrapa,np.NaN,np.NaN,np.NaN,[M.Bieb.],M.Bieb.,Centaurea,calcitrapa,-1,No Mout. author
301,Centaurea,calcitrapa,np.NaN,np.NaN,np.NaN,[L.],L.,Centaurea,calcitrapa,-1,No Mout. author
338,Centaurea,regia,regia,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,regia,-1,No author
344,Centaurea,solstitialis,solstitialis,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,solstitialis,-1,No author
356,Centaurea,virgata,virgata,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,virgata,-1,No author


In [345]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == "No author") |
                              (vol3_merge_simplified_grouped["author_match_type"] == "No Mout. author") |
                              (vol3_merge_simplified_grouped["author_match_type"] == 'No WCVP author') 
                             ].shape

(38, 11)

In [346]:
vol3_merge_simplified_grouped["author_match_type"].unique()

array(['type 0', 'type 3', 'No author', 'type 2', 'type -1', 'type 1',
       'No Mout. author', 'NO TYPE', 'No WCVP author'], dtype=object)

In [347]:
vol1_merge_simplified["author_match_type"].unique()

array(['No Mout. author', 'type 0', 'type -1', 'type 1', 'type 3',
       'type 2'], dtype=object)

In [348]:
vol1_merge_simplified

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
1,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,[C.A.Mey.],C.A.Mey.,Acanthophyllum,np.NaN,-1,No Mout. author
2,Acanthophyllum,kurdicum,np.NaN,Boiss. et Hausskn.,"[Boiss., Hausskn.]","[Boiss., Hausskn.]",Boiss. & Hausskn.,Acanthophyllum,kurdicum,2,type 0
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
4,Achyranthes,aspera,np.NaN,L.,[L.],[L.],L.,Achyranthes,aspera,1,type 0
...,...,...,...,...,...,...,...,...,...,...,...
1454,Vulpia,uniglumis,np.NaN,(Sol.) Dumort.,"[Sol., Dumort.]",[Dumort.],Dumort.,Vulpia,uniglumis,1,type 2
1455,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author
1456,Zannichellia,palustris,np.NaN,L.,[L.],[L.],L.,Zannichellia,palustris,1,type 0
1457,Zostera,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Zostera,np.NaN,-1,No Mout. author


In [349]:
vol1_merge_simplified_grouped

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
2,Acanthophyllum,kurdicum,np.NaN,Boiss. et Hausskn.,"[Boiss., Hausskn.]","[Boiss., Hausskn.]",Boiss. & Hausskn.,Acanthophyllum,kurdicum,2,type 0
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
4,Achyranthes,aspera,np.NaN,L.,[L.],[L.],L.,Achyranthes,aspera,1,type 0
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1454,Vulpia,uniglumis,np.NaN,(Sol.) Dumort.,"[Sol., Dumort.]",[Dumort.],Dumort.,Vulpia,uniglumis,1,type 2
1455,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author
1456,Zannichellia,palustris,np.NaN,L.,[L.],[L.],L.,Zannichellia,palustris,1,type 0
1457,Zostera,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Zostera,np.NaN,-1,No Mout. author


In [354]:
vol1_merge_simplified_grouped[vol1_merge_simplified_grouped['author_match_type'] == 'type -1']

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
29,Aegilotricum,loreti,np.NaN,(Richt.) Maire et Weiller,"[Richt., Maire, Weiller]",[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,0,type -1
48,Alisma,gramineum,np.NaN,Gmel.,[Gmel.],[Lej.],Lej.,Alisma,gramineum,0,type -1
49,Alisma,graminifolium,np.NaN,Ehrh.,[Ehrh.],[Glück],Glück,Alisma,graminifolium,0,type -1
50,Alisma,lanceolatum,np.NaN,Whit.,[Whit.],[With.],With.,Alisma,lanceolatum,0,type -1
60,Allium,aschersonianum,np.NaN,W. Barb.,[W. Barb.],[Barbey],Barbey,Allium,aschersonianum,0,type -1
...,...,...,...,...,...,...,...,...,...,...,...
1405,Tulipa,agenensis,np.NaN,D.C.,[D.C.],[Redouté],Redouté,Tulipa,agenensis,0,type -1
1406,Tulipa,aleppensis,np.NaN,Regel,[Regel],[Boiss.],Boiss. ex Regel,Tulipa,aleppensis,0,type -1
1408,Tulipa,biflora,np.NaN,Pali.,[Pali.],[Pall.],Pall.,Tulipa,biflora,0,type -1
1410,Tulipa,montana,np.NaN,Lindley,[Lindley],[Lindl.],Lindl.,Tulipa,montana,0,type -1


In [358]:
genus = "Alisma"
epithet = "gramineum"
infra = np.NaN
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet)]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
22305,77097302-1,Alismataceae,Alisma,gramineum,NaN,Alisma gramineum,Lej.,SPECIES,Accepted,NaN,NaN,NaN,328557-2,Alisma,L.,Reviewed,Fl. Spa 1: 175 (1811),NaN,[Lej.]
22306,874063-1,Alismataceae,Alisma,gramineum,aestuosum,Alisma gramineum f. aestuosum,(Bolle) Soó,Form,Synonym,77097302-1,Alisma gramineum,Lej.,NaN,NaN,NaN,Reviewed,Acta Bot. Acad. Sci. Hung. 16: 365 (1971),NaN,"[Bolle, Soó]"
22307,8479-2,Alismataceae,Alisma,gramineum,angustissimum,Alisma gramineum var. angustissimum,(DC.) Hendricks,VARIETY,Synonym,77097302-1,Alisma gramineum,Lej.,NaN,NaN,NaN,Reviewed,Amer. Midl. Naturalist 58: 489 (1957),8491-2,"[DC., Hendricks]"
22308,8480-2,Alismataceae,Alisma,gramineum,geyeri,Alisma gramineum var. geyeri,(Torr.) Sam.,VARIETY,Synonym,77097302-1,Alisma gramineum,Lej.,NaN,NaN,NaN,Reviewed,Ark. Bot. 24A(7): 43 (1932),58204-1,"[Torr., Sam.]"
22309,8482-2,Alismataceae,Alisma,gramineum,graminifolium,Alisma gramineum var. graminifolium,(Wahlenb.) Hendricks,VARIETY,Synonym,60449258-2,Alisma wahlenbergii,(Holmb.) Juz.,NaN,NaN,NaN,Reviewed,Amer. Midl. Naturalist 58: 487 (1957),8494-2,"[Wahlenb., Hendricks]"
22310,1160212-2,Alismataceae,Alisma,gramineum,wahlenbergii,Alisma gramineum subsp. wahlenbergii,Holmb.,SUBSPECIES,Homotypic_Synonym,60449258-2,Alisma wahlenbergii,(Holmb.) Juz.,NaN,NaN,NaN,Reviewed,Bot. Not. 1922: 207 (1922),NaN,[Holmb.]
22311,8483-2,Alismataceae,Alisma,gramineum,wahlenbergii,Alisma gramineum var. wahlenbergii,(Holmb.) Raymond & Kucyn.,VARIETY,Homotypic_Synonym,60449258-2,Alisma wahlenbergii,(Holmb.) Juz.,NaN,NaN,NaN,Reviewed,Rhodora 50: 179 (1948),1160212-2,"[Holmb., Raymond, Kucyn.]"


##### Vol1 No Author

In [362]:
vol1_merge_simplified_grouped[(vol1_merge_simplified_grouped["author_match_type"] == "No author") |
                              (vol1_merge_simplified_grouped["author_match_type"] == "No Mout. author") |
                              (vol1_merge_simplified_grouped["author_match_type"] == 'No WCVP author') 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
7,Aegilops,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Aegilops,np.NaN,-1,No Mout. author
28,Aegilotricum,np.NaN,np.NaN,np.NaN,np.NaN,[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1440,Velezia,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Velezia,np.NaN,-1,No Mout. author
1443,Viscum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Viscum,np.NaN,-1,No Mout. author
1446,Vulpia,np.NaN,np.NaN,np.NaN,np.NaN,[C.C.Gmel.],C.C.Gmel.,Vulpia,np.NaN,-1,No Mout. author
1455,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author


In [363]:
vol1_merge_simplified_grouped[(vol1_merge_simplified_grouped["author_match_type"] == "No author") 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [366]:
vol1_merge_simplified_grouped[(vol1_merge_simplified_grouped["author_match_type"] == 'No WCVP author')]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [365]:
vol1_merge_simplified_grouped[(vol1_merge_simplified_grouped["author_match_type"] == "No Mout. author")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
7,Aegilops,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Aegilops,np.NaN,-1,No Mout. author
28,Aegilotricum,np.NaN,np.NaN,np.NaN,np.NaN,[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1440,Velezia,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Velezia,np.NaN,-1,No Mout. author
1443,Viscum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Viscum,np.NaN,-1,No Mout. author
1446,Vulpia,np.NaN,np.NaN,np.NaN,np.NaN,[C.C.Gmel.],C.C.Gmel.,Vulpia,np.NaN,-1,No Mout. author
1455,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author


In [373]:
vol1_merge_simplified[(vol1_merge_simplified["author_match_type"] == "No author") |
                              (vol1_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol1_merge_simplified["author_match_type"] == 'No WCVP author') 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,"[Hook., Arn.]",Hook. & Arn.,Acanthophyllum,np.NaN,-1,No Mout. author
1,Acanthophyllum,np.NaN,np.NaN,np.NaN,np.NaN,[C.A.Mey.],C.A.Mey.,Acanthophyllum,np.NaN,-1,No Mout. author
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
7,Aegilops,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Aegilops,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1440,Velezia,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Velezia,np.NaN,-1,No Mout. author
1443,Viscum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Viscum,np.NaN,-1,No Mout. author
1446,Vulpia,np.NaN,np.NaN,np.NaN,np.NaN,[C.C.Gmel.],C.C.Gmel.,Vulpia,np.NaN,-1,No Mout. author
1455,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author


##### Vol2 No Author

In [367]:
vol2_merge_simplified_grouped[(vol2_merge_simplified_grouped["author_match_type"] == "No author") |
                              (vol2_merge_simplified_grouped["author_match_type"] == "No Mout. author") |
                              (vol2_merge_simplified_grouped["author_match_type"] == 'No WCVP author') 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acacia,np.NaN,np.NaN,np.NaN,np.NaN,[Mill.],Mill.,Acacia,np.NaN,-1,No Mout. author
5,Acer,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Acer,np.NaN,-1,No Mout. author
9,Actinolema,np.NaN,np.NaN,np.NaN,np.NaN,[Fenzl],Fenzl,Actinolema,np.NaN,-1,No Mout. author
12,Adenocarpus,np.NaN,np.NaN,np.NaN,np.NaN,[DC.],DC.,Adenocarpus,np.NaN,-1,No Mout. author
14,Adonis,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adonis,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1542,Punica,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,np.NaN,-1,No Mout. author
1543,Punica,granatum,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,granatum,-1,No Mout. author
1544,Pyracantha,np.NaN,np.NaN,np.NaN,np.NaN,[M.Roem.],M.Roem.,Pyracantha,np.NaN,-1,No Mout. author
1546,Radiola,np.NaN,np.NaN,np.NaN,np.NaN,[Hill],Hill,Radiola,np.NaN,-1,No Mout. author


In [368]:
vol2_merge_simplified_grouped[(vol2_merge_simplified_grouped["author_match_type"] == "No author") 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [369]:
vol2_merge_simplified_grouped[(vol2_merge_simplified_grouped["author_match_type"] == 'No WCVP author')]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type


In [371]:
vol2_merge_simplified_grouped[(vol2_merge_simplified_grouped["author_match_type"] == "No Mout. author")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acacia,np.NaN,np.NaN,np.NaN,np.NaN,[Mill.],Mill.,Acacia,np.NaN,-1,No Mout. author
5,Acer,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Acer,np.NaN,-1,No Mout. author
9,Actinolema,np.NaN,np.NaN,np.NaN,np.NaN,[Fenzl],Fenzl,Actinolema,np.NaN,-1,No Mout. author
12,Adenocarpus,np.NaN,np.NaN,np.NaN,np.NaN,[DC.],DC.,Adenocarpus,np.NaN,-1,No Mout. author
14,Adonis,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adonis,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1542,Punica,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,np.NaN,-1,No Mout. author
1543,Punica,granatum,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,granatum,-1,No Mout. author
1544,Pyracantha,np.NaN,np.NaN,np.NaN,np.NaN,[M.Roem.],M.Roem.,Pyracantha,np.NaN,-1,No Mout. author
1546,Radiola,np.NaN,np.NaN,np.NaN,np.NaN,[Hill],Hill,Radiola,np.NaN,-1,No Mout. author


In [372]:
vol2_merge_simplified[(vol2_merge_simplified["author_match_type"] == "No author") |
                              (vol2_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol2_merge_simplified["author_match_type"] == 'No WCVP author') 
                             ]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acacia,np.NaN,np.NaN,np.NaN,np.NaN,[Mill.],Mill.,Acacia,np.NaN,-1,No Mout. author
5,Acer,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Acer,np.NaN,-1,No Mout. author
9,Actinolema,np.NaN,np.NaN,np.NaN,np.NaN,[Fenzl],Fenzl,Actinolema,np.NaN,-1,No Mout. author
12,Adenocarpus,np.NaN,np.NaN,np.NaN,np.NaN,[DC.],DC.,Adenocarpus,np.NaN,-1,No Mout. author
14,Adonis,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adonis,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1542,Punica,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,np.NaN,-1,No Mout. author
1543,Punica,granatum,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,granatum,-1,No Mout. author
1544,Pyracantha,np.NaN,np.NaN,np.NaN,np.NaN,[M.Roem.],M.Roem.,Pyracantha,np.NaN,-1,No Mout. author
1546,Radiola,np.NaN,np.NaN,np.NaN,np.NaN,[Hill],Hill,Radiola,np.NaN,-1,No Mout. author


##### Vol3 No Author

In [386]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == "No author") |
                              (vol3_merge_simplified_grouped["author_match_type"] == "No Mout. author") |
                              (vol3_merge_simplified_grouped["author_match_type"] == 'No WCVP author') 
                             ] #shape[0] 38

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
141,BallotaL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Ballota,np.NaN,-1,No Mout. author
268,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
278,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
299,Centaurea,bruguieriana,bruguieriana,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,bruguieriana,-1,No author
300,Centaurea,calcitrapa,np.NaN,np.NaN,np.NaN,[M.Bieb.],M.Bieb.,Centaurea,calcitrapa,-1,No Mout. author
338,Centaurea,regia,regia,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,regia,-1,No author
344,Centaurea,solstitialis,solstitialis,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,solstitialis,-1,No author
356,Centaurea,virgata,virgata,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,virgata,-1,No author
442,CordiaL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Cordia,np.NaN,-1,No Mout. author


In [393]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == "No author")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
268,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
278,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
299,Centaurea,bruguieriana,bruguieriana,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,bruguieriana,-1,No author
338,Centaurea,regia,regia,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,regia,-1,No author
344,Centaurea,solstitialis,solstitialis,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,solstitialis,-1,No author
356,Centaurea,virgata,virgata,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,virgata,-1,No author
464,Crépis,foetida,foetida,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,foetida,-1,No author
476,Crépis,sancta,sancta,np.NaN,np.NaN,np.NaN,np.NaN,Crepis,sancta,-1,No author
501,Cuscuta,approximata,approximata,np.NaN,np.NaN,np.NaN,np.NaN,Cuscuta,approximata,-1,No author


In [394]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == 'No WCVP author')]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
1640,Veronica,macrostachya,macrostachya,orientalis Miller,[orientalis Miller],np.NaN,np.NaN,Veronica,macrostachya,-1,No WCVP author


In [395]:
vol3_merge_simplified_grouped[(vol3_merge_simplified_grouped["author_match_type"] == "No Mout. author")]

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
141,BallotaL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Ballota,np.NaN,-1,No Mout. author
300,Centaurea,calcitrapa,np.NaN,np.NaN,np.NaN,[M.Bieb.],M.Bieb.,Centaurea,calcitrapa,-1,No Mout. author
442,CordiaL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Cordia,np.NaN,-1,No Mout. author
450,Cousinia,libanotica,np.NaN,np.NaN,np.NaN,[DC.],DC.,Cousinia,libanotica,-1,No Mout. author
620,Galium,canum,np.NaN,np.NaN,np.NaN,[Req.],Req. ex DC.,Galium,canum,-1,No Mout. author
654,Galium,thiebautii,np.NaN,np.NaN,np.NaN,[Ehrend.],Ehrend.,Galium,thiebautii,-1,No Mout. author
722,Hyoscyamus,albu^L.,np.NaN,np.NaN,np.NaN,[L.],L.,Hyoscyamus,albus,-1,No Mout. author
920,Myopordon,genitz,np.NaN,np.NaN,np.NaN,[Boiss.],Boiss.,Myopordon,np.NaN,-1,No Mout. author
1003,Origanum,bargyli,np.NaN,np.NaN,np.NaN,[Mouterde],Mouterde,Origanum,bargyli,-1,No Mout. author
1004,Origanum,syriacum,np.NaN,np.NaN,np.NaN,[L.],L.,Origanum,syriacum,-1,No Mout. author


In [396]:
vol3_merge_simplified[(vol3_merge_simplified["author_match_type"] == "No author") |
                              (vol3_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol3_merge_simplified["author_match_type"] == 'No WCVP author') ] #shape[0] 48

,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
141,BallotaL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Ballota,np.NaN,-1,No Mout. author
268,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
278,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
299,Centaurea,bruguieriana,bruguieriana,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,bruguieriana,-1,No author
300,Centaurea,calcitrapa,np.NaN,np.NaN,np.NaN,[M.Bieb.],M.Bieb.,Centaurea,calcitrapa,-1,No Mout. author
301,Centaurea,calcitrapa,np.NaN,np.NaN,np.NaN,[L.],L.,Centaurea,calcitrapa,-1,No Mout. author
338,Centaurea,regia,regia,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,regia,-1,No author
344,Centaurea,solstitialis,solstitialis,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,solstitialis,-1,No author
356,Centaurea,virgata,virgata,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,virgata,-1,No author


##### # NOTE: for all the no Mout. Author types the author may be wrong have and have multiple possibilities!!! 

but that's not the case for the majority fo them which is good! 


In [ ]:
#vol1_merge_simplified = vol1_merge_simplified.loc[vol1_merge_simplified.groupby(['mout_genus_close_match', 'mout_epithet_close_match', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]#[['bookid', 'conceptid', 'weight']]
vol1_merge_simplified_grouped = vol1_merge_simplified.loc[vol1_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]
vol2_merge_simplified_grouped = vol2_merge_simplified.loc[vol2_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]
vol3_merge_simplified_grouped = vol3_merge_simplified.loc[vol3_merge_simplified.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author'], sort=False)['num_author_overlap'].idxmax()]

In [425]:
vol1_no_author_combos = vol1_merge_simplified[(vol1_merge_simplified["author_match_type"] == "No author") |
                              (vol1_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol1_merge_simplified["author_match_type"] == 'No WCVP author')]

vol1_no_author_combos.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author']).filter(lambda x  : len(x) == 1)


,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
3,Achyranthes,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Achyranthes,np.NaN,-1,No Mout. author
5,Adiantum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adiantum,np.NaN,-1,No Mout. author
7,Aegilops,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Aegilops,np.NaN,-1,No Mout. author
28,Aegilotricum,np.NaN,np.NaN,np.NaN,np.NaN,[R.Wagner],R.Wagner ex Tscherm.-Seys.,Aegilotricum,np.NaN,-1,No Mout. author
30,Aellenia,np.NaN,np.NaN,np.NaN,np.NaN,[Ulbr.],Ulbr.,Aellenia,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1440,Velezia,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Velezia,np.NaN,-1,No Mout. author
1443,Viscum,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Viscum,np.NaN,-1,No Mout. author
1446,Vulpia,np.NaN,np.NaN,np.NaN,np.NaN,[C.C.Gmel.],C.C.Gmel.,Vulpia,np.NaN,-1,No Mout. author
1455,Zannichellia,np.NaN,np.NaN,np.NaN,np.NaN,[P.Micheli],P.Micheli ex L.,Zannichellia,np.NaN,-1,No Mout. author


In [429]:
vol1_no_author_combos.shape[0]

328

In [428]:
vol2_no_author_combos = vol2_merge_simplified[(vol2_merge_simplified["author_match_type"] == "No author") |
                              (vol2_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol2_merge_simplified["author_match_type"] == 'No WCVP author')]

vol2_no_author_combos.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author']).filter(lambda x  : len(x) == 1)


,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
0,Acacia,np.NaN,np.NaN,np.NaN,np.NaN,[Mill.],Mill.,Acacia,np.NaN,-1,No Mout. author
5,Acer,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Acer,np.NaN,-1,No Mout. author
9,Actinolema,np.NaN,np.NaN,np.NaN,np.NaN,[Fenzl],Fenzl,Actinolema,np.NaN,-1,No Mout. author
12,Adenocarpus,np.NaN,np.NaN,np.NaN,np.NaN,[DC.],DC.,Adenocarpus,np.NaN,-1,No Mout. author
14,Adonis,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Adonis,np.NaN,-1,No Mout. author
...,...,...,...,...,...,...,...,...,...,...,...
1542,Punica,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,np.NaN,-1,No Mout. author
1543,Punica,granatum,np.NaN,np.NaN,np.NaN,[L.],L.,Punica,granatum,-1,No Mout. author
1544,Pyracantha,np.NaN,np.NaN,np.NaN,np.NaN,[M.Roem.],M.Roem.,Pyracantha,np.NaN,-1,No Mout. author
1546,Radiola,np.NaN,np.NaN,np.NaN,np.NaN,[Hill],Hill,Radiola,np.NaN,-1,No Mout. author


In [430]:
vol2_no_author_combos.shape[0]

334

In [434]:
vol3_no_author_combos = vol3_merge_simplified[(vol3_merge_simplified["author_match_type"] == "No author") |
                              (vol3_merge_simplified["author_match_type"] == "No Mout. author") |
                              (vol3_merge_simplified["author_match_type"] == 'No WCVP author')]

vol3_no_author_combos.groupby(['mouterde_genus', 'mouterde_epithet', 'mouterde_infra', 'mouterde_author']).filter(lambda x  : len(x) == 1) #.shape[0] 34


,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,mouterde_author_list,author_list,authors,genus,species,num_author_overlap,author_match_type
14,Achillea,aleppica,aleppica,np.NaN,np.NaN,np.NaN,np.NaN,Achillea,aleppica,-1,No author
141,BallotaL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Ballota,np.NaN,-1,No Mout. author
268,Carlina,oligocephala,oligocephala,np.NaN,np.NaN,np.NaN,np.NaN,Carlina,oligocephala,-1,No author
278,Carthamus,tenuis,tenuis,np.NaN,np.NaN,np.NaN,np.NaN,Carthamus,tenuis,-1,No author
299,Centaurea,bruguieriana,bruguieriana,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,bruguieriana,-1,No author
338,Centaurea,regia,regia,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,regia,-1,No author
344,Centaurea,solstitialis,solstitialis,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,solstitialis,-1,No author
356,Centaurea,virgata,virgata,np.NaN,np.NaN,np.NaN,np.NaN,Centaurea,virgata,-1,No author
442,CordiaL.,np.NaN,np.NaN,np.NaN,np.NaN,[L.],L.,Cordia,np.NaN,-1,No Mout. author
450,Cousinia,libanotica,np.NaN,np.NaN,np.NaN,[DC.],DC.,Cousinia,libanotica,-1,No Mout. author


In [432]:
vol3_no_author_combos.shape[0]

48

In [443]:
genus = "Xanthium"#"Achillea"
epithet = "strumarium"#"aleppica"
infra = "strumarium"#"aleppica"
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'] == epithet)]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
273654,77194547-1,Asteraceae,Xanthium,strumarium,strumarium,Xanthium strumarium subsp. strumarium,NaN,SUBSPECIES,Synonym,260893-1,Xanthium strumarium,L.,NaN,NaN,NaN,In review,NaN,NaN,NaN


In [441]:
genus = "Galium"
epithet = "canum"
wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'].isna())]

,kew_id,family,genus,species,infraspecies,taxon_name,authors,rank,taxonomic_status,accepted_kew_id,accepted_name,accepted_authors,parent_kew_id,parent_name,parent_authors,reviewed,publication,original_name_id,author_list
1112424,749676-1,Rubiaceae,Galium,canum,NaN,Galium canum,Req. ex DC.,SPECIES,Accepted,NaN,NaN,NaN,325955-2,Galium,L.,Reviewed,Prodr. 4: 602 (1830),NaN,[Req.]


## Untitled